BoW vs TfIdf

In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path
import re
import string
import pandas as pd
import mlflow
import dagshub
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import scipy.sparse

In [6]:
load_dotenv()

REPO_OWNER = os.getenv("REPO_OWNER")
REPO_NAME = os.getenv("REPO_NAME")
dagshub.init(repo_owner=REPO_OWNER, repo_name=REPO_NAME, mlflow=True)

data_path = Path.cwd().parent / "data" / "data.csv"

MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

Initialized MLflow to track repo "Shreyaan16/Sentiment-Analysis-IMDB"

Repository Shreyaan16/Sentiment-Analysis-IMDB initialized!

In [17]:
mlflow.set_experiment("BoW vs TfIdf")

2026/01/11 19:21:17 INFO mlflow.tracking.fluent: Experiment with name 'BoW vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/370681dbf2064817b063947b8ffa82ad', creation_time=1768139479213, experiment_id='1', last_update_time=1768139479213, lifecycle_stage='active', name='BoW vs TfIdf', tags={}>

In [7]:
df = pd.read_csv(data_path)
df.head()

,review,sentiment
0,Bad. Personal opinion? The folks who made it? ...,negative
1,This movie is obviously low-budget & filmed in...,positive
2,"Yes, this movie has kids going to space camp a...",negative
3,"Before I begin, let me tell you how GREAT this...",positive
4,The Vampire Bat is set in the small German vil...,negative


In [8]:
def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

In [9]:
def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

In [10]:
def lower_case(text):
    """Convert text to lower case."""
    return text.lower()

In [11]:
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

In [12]:
def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = text.replace(';', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [13]:
def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [14]:
def normalize_text(df):
    """Normalize the text data."""
    df['review'] = df['review'].apply(lower_case)
    df['review'] = df['review'].apply(remove_stop_words)
    df['review'] = df['review'].apply(removing_numbers)
    df['review'] = df['review'].apply(removing_punctuations)
    df['review'] = df['review'].apply(removing_urls)
    df['review'] = df['review'].apply(lemmatization)
    return df

In [15]:
df = normalize_text(df)
df.head()

,review,sentiment
0,bad personal opinion folk made it knew made it...,negative
1,movie obviously low budget filmed british colu...,positive
2,yes movie kid going space camp start okay enou...,negative
3,begin let tell great stand up special sound pl...,positive
4,vampire bat set small german village klineschl...,negative


In [16]:
sentiment_mapper = {"positive" : 1 , "negative" : 0}
df['sentiment'] = df['sentiment'].map(sentiment_mapper)
df.head()

,review,sentiment
0,bad personal opinion folk made it knew made it...,0
1,movie obviously low budget filmed british colu...,1
2,yes movie kid going space camp start okay enou...,0
3,begin let tell great stand up special sound pl...,1
4,vampire bat set small german village klineschl...,0


In [18]:
VECTORIZERS = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

In [19]:
ALGORITHMS = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [20]:
def log_model_params(algo_name, model):
    """
    Logs hyperparameters of the trained model to MLflow.
    """
    params_to_log = {}
    if algo_name == 'LogisticRegression':
        params_to_log["C"] = model.C
    elif algo_name == 'MultinomialNB':
        params_to_log["alpha"] = model.alpha
    elif algo_name == 'XGBoost':
        params_to_log["n_estimators"] = model.n_estimators
        params_to_log["learning_rate"] = model.learning_rate
    elif algo_name == 'RandomForest':
        params_to_log["n_estimators"] = model.n_estimators
        params_to_log["max_depth"] = model.max_depth
    elif algo_name == 'GradientBoosting':
        params_to_log["n_estimators"] = model.n_estimators
        params_to_log["learning_rate"] = model.learning_rate
        params_to_log["max_depth"] = model.max_depth
    mlflow.log_params(params_to_log)


In [21]:
def train_and_evaluate(df):
    with mlflow.start_run(run_name="All Experiments") as parent_run:
        for algo_name, algorithm in ALGORITHMS.items():
            for vec_name, vectorizer in VECTORIZERS.items():
                with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                    try:
                        # Feature extraction
                        X = vectorizer.fit_transform(df['review'])
                        y = df['sentiment']
                        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                        # Log preprocessing parameters
                        mlflow.log_params({
                            "vectorizer": vec_name,
                            "algorithm": algo_name,
                            "test_size": 0.2
                        })

                        # Train model
                        model = algorithm
                        model.fit(X_train, y_train)

                        # Log model parameters
                        log_model_params(algo_name, model)

                        # Evaluate model
                        y_pred = model.predict(X_test)
                        metrics = {
                            "accuracy": accuracy_score(y_test, y_pred),
                            "precision": precision_score(y_test, y_pred),
                            "recall": recall_score(y_test, y_pred),
                            "f1_score": f1_score(y_test, y_pred)
                        }
                        mlflow.log_metrics(metrics)

                        # Log model
                        input_example = X_test[:5] if not scipy.sparse.issparse(X_test) else X_test[:5].toarray()
                        mlflow.sklearn.log_model(model, "model", input_example=input_example)

                        # Print results for verification
                        print(f"\nAlgorithm: {algo_name}, Vectorizer: {vec_name}")
                        print(f"Metrics: {metrics}")

                    except Exception as e:
                        print(f"Error in training {algo_name} with {vec_name}: {e}")
                        mlflow.log_param("error", str(e))

In [22]:
train_and_evaluate(df)

2026/01/11 20:27:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Shreyaan\anaconda3\envs\mlops_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Algorithm: LogisticRegression, Vectorizer: BoW
Metrics: {'accuracy': 0.73, 'precision': 0.75, 'recall': 0.7211538461538461, 'f1_score': 0.7352941176470589}
🏃 View run LogisticRegression with BoW at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1/runs/17fdbd24f50d4b92ba007240381030df
🧪 View experiment at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1


2026/01/11 20:27:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Algorithm: LogisticRegression, Vectorizer: TF-IDF
Metrics: {'accuracy': 0.77, 'precision': 0.7959183673469388, 'recall': 0.75, 'f1_score': 0.7722772277227723}
🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1/runs/4a18ed70f0da48b3a080690eb457a97b
🧪 View experiment at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1


2026/01/11 20:28:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...


Algorithm: RandomForest, Vectorizer: TF-IDF
Metrics: {'accuracy': 0.745, 'precision': 0.7623762376237624, 'recall': 0.7403846153846154, 'f1_score': 0.751219512195122}
🏃 View run RandomForest with TF-IDF at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1/runs/d982012c4e2a402c9c4943c0e9ffa5b3
🧪 View experiment at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1


2026/01/11 20:30:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Algorithm: GradientBoosting, Vectorizer: BoW
Metrics: {'accuracy': 0.76, 'precision': 0.7745098039215687, 'recall': 0.7596153846153846, 'f1_score': 0.7669902912621359}
🏃 View run GradientBoosting with BoW at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1/runs/02a8284c55c6494ebfcaa9b3abafc76e
🧪 View experiment at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1


2026/01/11 20:31:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Algorithm: GradientBoosting, Vectorizer: TF-IDF
Metrics: {'accuracy': 0.745, 'precision': 0.7431192660550459, 'recall': 0.7788461538461539, 'f1_score': 0.7605633802816901}
🏃 View run GradientBoosting with TF-IDF at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1/runs/bb17b173d3d442669f2e655417e55ef7
🧪 View experiment at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1
🏃 View run All Experiments at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1/runs/feb6733796f9431bbcfd0f2aaefa6225
🧪 View experiment at: https://dagshub.com/Shreyaan16/Sentiment-Analysis-IMDB.mlflow/#/experiments/1
